# Stand-up comedy topic modeling across cultures
## Natural Language Processing (E1282) Research Note, Dr. Sascha Göbel
- Author: Lonny Chen (216697)
- Submission Date: 5 January 2026

In [1]:
# Local environment
import sys
sys.path.append('../../')
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from data.data_collection import get_channels_shorts_video_ids, get_transcript_from_video_id

## Data Collection
### Step 1: Get YouTube video IDs
- Use of [YT-DLP](https://github.com/yt-dlp/yt-dlp) for extracting video information is referenced from [StandUp4AI](https://github.com/Standup4AI/dataset/) project code.

In [3]:
%%time
# Input YouTube channel search information here
channels = [
    {
        'channel': 'Comedy Central Stand-Up',
        'search_url': 'https://www.youtube.com/@standup/shorts',
        'list_search_str': [],
        'language_code': 'en'
    },
    {
        'channel': 'Comedy Central UK',
        'search_url': 'https://www.youtube.com/@comedycentraluk/search?query=standup',
        'list_search_str': [],
        'language_code': 'en'
    }
]
df_channels = get_channels_shorts_video_ids(channels=channels)
#df_channels.to_csv('data/df_channels.csv', index=False)

Channel @standup extracted 404 videos from https://www.youtube.com/@standup/shorts
Channel @comedycentraluk extracted 375 videos from https://www.youtube.com/@comedycentraluk/search?query=standup
CPU times: user 2.83 s, sys: 78.6 ms, total: 2.91 s
Wall time: 13.3 s


/Users/lonnychen/Dropbox/@project - HERTIE SCHOOL/3. NLP (E1282)/A3. Research Note/nlp_research_note/data/data_collection.py:108: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(df_channels_list)


### Step 2: Get transcript strings from video IDs
- Use [YouTube Transcript API](https://github.com/jdepoix/youtube-transcript-api) to fetch correct language transcripts
- Internally uses "Rotating Residential Proxy" from [Webshare](https://www.webshare.io/?referral_code=w0xno53eb50g) to workaround <span style="color:red">IpBlocked</span> exceptions

In [4]:
%%time
# TEST: one transcript fetch
video_id = 'JYjRBzuVFwQ' #test shorts
get_transcript_from_video_id(video_id, 'en', use_proxy=True)

# TEST: various tasks using small sample of videos
#df_test = df_channels[0:5].copy()
#df_test['transcript'] = df_test.apply(lambda row: get_transcript_from_video_id(row['video_id'], row['language_code']), axis=1)

Fetch completed for video 'JYjRBzuVFwQ'
CPU times: user 108 ms, sys: 21.1 ms, total: 129 ms
Wall time: 6.36 s


"it was great being in a relationship\nwith her but I didn't know like Latino\nwomen like I didn't know that like\nyou're under investigation like it's\njust when you date a Puerto Rican girl\nyou need to have the answers quick you\nneed to like have facts right away\nbecause they will interrogate you and if\nyou don't have the answers quick you\nlook like a liar and they watch men they\nare designed to watch men watching\nwaiting that's what they do they make\nexcellent NFL referees I think if you\nhad Puerto Rican girls reing the games\nyou'd have zero Miss calls the whole\nseason because they see everything so\nthat's just what could you imagine they\nwere the refs they would be right there\nlike um you out of bound stupid no\nno I mean for real you out of\nbounds I don't need instant replay I I\ninstantly saw you step out of bound so\nyou\nknow I mean I don't even know the rules\njust go home\nbye you out the game"

In [ ]:
%%time
# FULL set of videos (2-3 hours for n=~770)
df_channels['transcript'] = df_channels.apply(lambda row: get_transcript_from_video_id(row['video_id'], row['language_code']), axis=1)
# Save to CSV
df_channels.to_csv('data/df_channels_transcripts.csv', index=False)

In [5]:
# Read from saved CSV
df_channels_load = pd.read_csv('data/df_channels_transcripts.csv')
df_channels_load.shape

(768, 9)